In [ ]:
import pandas as pd
import os
import glob
from tqdm import tqdm

# =============================================================================
# [설정] 검색 시작 루트 경로 (이 경로 하위의 모든 폴더를 뒤집니다)
# =============================================================================
BASE_DIR = "/home/jovyan/CHJ/Mol-LLM_Custom/Inference_log/Benchmark_inference_csv"

# 검사할 종료 토큰
TARGET_TOKEN = "<|eot_id|>"

def check_all_csv_files():
    # 1. 모든 CSV 파일 찾기 (재귀적 탐색)
    print(f"🔍 Searching for CSV files in: {BASE_DIR} ...")
    all_csv_files = glob.glob(os.path.join(BASE_DIR, "**/*.csv"), recursive=True)
    
    # 이미 생성된 에러 리포트 파일은 제외 (파일명에 '_error'가 포함된 경우 등)
    target_files = [f for f in all_csv_files if "_no_eot_error" not in f and "checkpoint" not in f]
    
    print(f"found {len(target_files)} CSV files to check.\n")

    summary_report = []

    # 2. 파일 순회하며 검사
    for file_path in tqdm(target_files, desc="Checking files"):
        try:
            # 로딩 속도를 위해 'label' 컬럼만 로드 (없으면 전체 로드 시도 후 에러 처리)
            try:
                df = pd.read_csv(file_path, usecols=['label'])
            except ValueError:
                # label 컬럼이 없는 파일일 수 있음 (전체 읽어서 확인)
                df = pd.read_csv(file_path)
                if 'label' not in df.columns:
                    continue  # label 컬럼 없으면 스킵

            total_rows = len(df)
            if total_rows == 0:
                continue

            # 3. 검사 로직 (공백 제거 후 토큰 확인)
            # NaN은 문자열 변환 -> 공백 제거 -> endswith 확인 -> 부정(~)
            mask = ~df['label'].astype(str).str.strip().str.endswith(TARGET_TOKEN)
            error_count = mask.sum()

            status = "✅ Pass"
            saved_path = "-"

            # 4. 문제가 있다면 별도 파일로 저장
            if error_count > 0:
                status = f"❌ {error_count} Errors"
                
                # 전체 데이터를 다시 읽어와서(컬럼 전체) 문제 행 저장
                full_df = pd.read_csv(file_path)
                problematic_df = full_df[mask]
                
                output_path = file_path.replace(".csv", "_no_eot_error.csv")
                problematic_df.to_csv(output_path, index=False, encoding='utf-8-sig')
                saved_path = os.path.basename(output_path)

            # 리포트에 추가
            summary_report.append({
                "File Name": os.path.basename(file_path),
                "Total Rows": total_rows,
                "Missing EOT": error_count,
                "Status": status,
                "Saved File": saved_path,
                "Full Path": file_path
            })

        except Exception as e:
            print(f"\n[Error] Failed to process {os.path.basename(file_path)}: {e}")

    # 5. 최종 결과 출력
    if summary_report:
        result_df = pd.DataFrame(summary_report)
        
        # 보기 좋게 출력하기 위해 정렬 (에러 많은 순)
        result_df = result_df.sort_values(by="Missing EOT", ascending=False)

        print("\n" + "="*80)
        print("📊 [Check Summary Report]")
        print("="*80)
        
        # 주요 컬럼만 출력
        print(result_df[['File Name', 'Total Rows', 'Missing EOT', 'Status']].to_string(index=False))
        
        print("="*80)
        
        # 에러가 있는 파일들만 모아서 경로 출력
        error_files = result_df[result_df['Missing EOT'] > 0]
        if not error_files.empty:
            print(f"\n⚠️ Total {len(error_files)} files have missing '{TARGET_TOKEN}' issues.")
        else:
            print(f"\n✅ All checked files are clean! (All labels end with '{TARGET_TOKEN}')")
            
    else:
        print("No CSV files processed.")

if __name__ == "__main__":
    check_all_csv_files()

SyntaxError: invalid syntax (1967084579.py, line 102)

In [ ]:
from datasets import load_from_disk
from collections import Counter
import pandas as pd
import os
import glob
from tqdm import tqdm

test_ds = load_from_disk('/home/jovyan/CHJ/Mol-LLM_Custom/dataset/train_official/GSAI-ML-LLaDA-8B-Instruct_string+graph_q32_test_512_Truncation')
test_smol_generation_ds = test_ds.filter(lambda x:  '<SELFIES>[C][C][=Branch1][C][=O][N][C@H1][C@H1][Branch2][Ring1][#Branch2][O][C@H1][C@H1][Branch1][C][O][C@@H1][Branch1][#Branch1][N][C][Branch1][C][C][=O]' in x['target_text'])

Filter:   0%|          | 0/32595 [00:00<?, ? examples/s]

In [5]:
test_smol_generation_ds[0]

{'x': [[5, 0, 4, 5, 3, 0, 2, 0, 0], [5, 0, 4, 5, 3, 0, 2, 0, 0]],
 'edge_index': [[0, 1], [1, 0]],
 'edge_attr': [[0, 0, 0], [0, 0, 0]],
 'label': '<SELFIES>[C][C][=Branch1][C][=O][N][C@H1][C@H1][Branch2][Ring1][#Branch2][O][C@H1][C@H1][Branch1][C][O][C@@H1][Branch1][#Branch1][N][C][Branch1][C][C][=O][C][Branch1][C][O][O][C@@H1][Ring1][N][C][O][O][C@H1][Branch1][Ring1][C][O][C@@H1][Branch2][Branch2][#Branch1][O][C@@H1][O][C@H1][Branch2][Branch1][#C][C][O][C@H1][O][C@H1][Branch2][Ring1][=Branch1][C][O][C@H1][O][C@H1][Branch1][Ring1][C][O][C@@H1][Branch1][C][O][C@H1][Branch1][C][O][C@@H1][Ring1][#Branch2][O][C@@H1][Branch1][C][O][C@H1][Branch2][Ring1][Branch1][O][C@H1][O][C@H1][Branch1][Ring1][C][O][C@@H1][Branch1][C][O][C@H1][Branch1][C][O][C@@H1][Ring1][#Branch2][O][C@@H1][Ring2][Ring1][S][O][C@@H1][Branch1][C][O][C@H1][Branch2][Ring1][Branch1][O][C@H1][O][C@H1][Branch1][Ring1][C][O][C@@H1][Branch1][C][O][C@H1][Branch1][C][O][C@@H1][Ring1][#Branch2][O][C@@H1][Ring2][Branch1][=Branch1][

# Benchmark 데이터셋 검증

## label의 끝이 `<|eot_id|>`이 아닌 sample csv 저장

In [10]:
import pandas as pd
import os
import glob
import json
from tqdm import tqdm

# =============================================================================
# [설정] 경로 설정
# =============================================================================
SEARCH_DIR = "/home/jovyan/CHJ/Mol-LLM_Custom/Inference_log/Benchmark_inference_csv"
SAVE_DIR = "/home/jovyan/CHJ/Mol-LLM_Custom/utils"

TARGET_TOKEN = "<|eot_id|>"

# 저장할 파일명 정의
OUTPUT_JSON_NAME = "all_missing_eot_errors.json"
OUTPUT_CSV_NAME = "all_missing_eot_errors.csv"

def save_merged_error_samples():
    # 저장 디렉토리 생성
    os.makedirs(SAVE_DIR, exist_ok=True)
    
    print(f"🔍 Searching for CSV files in: {SEARCH_DIR} ...")
    
    # 1. 모든 CSV 파일 재귀 탐색
    all_csv_files = glob.glob(os.path.join(SEARCH_DIR, "**/*.csv"), recursive=True)
    
    # 결과 파일, 체크포인트 등 제외
    target_files = [
        f for f in all_csv_files 
        if "_error" not in f and "checkpoint" not in f
    ]
    
    print(f"found {len(target_files)} CSV files to check.\n")

    # 모든 에러 데이터를 담을 리스트
    all_error_dfs = []

    # 2. 파일 순회하며 검사
    for file_path in tqdm(target_files, desc="Scanning files"):
        try:
            # CSV 읽기
            try:
                df = pd.read_csv(file_path)
            except Exception:
                continue 

            if 'label' not in df.columns:
                continue

            # 3. 에러 데이터 필터링 (공백 제거 후 토큰 확인)
            mask = ~df['label'].astype(str).str.strip().str.endswith(TARGET_TOKEN)
            
            error_df = df[mask].copy()

            # 4. 에러가 있다면 리스트에 추가
            if not error_df.empty:
                # [중요] 어떤 파일에서 온 에러인지 출처 기록
                error_df['source_file'] = os.path.basename(file_path)
                
                # 원본 csv의 전체 경로도 필요하면 아래 주석 해제
                # error_df['source_path'] = file_path 
                
                all_error_dfs.append(error_df)

        except Exception as e:
            print(f"⚠️ Error processing {os.path.basename(file_path)}: {e}")

    # 5. 하나로 병합 및 저장
    print("\n" + "="*80)
    
    if all_error_dfs:
        # 모든 에러 DataFrame 하나로 합치기
        merged_df = pd.concat(all_error_dfs, ignore_index=True)
        
        total_errors = len(merged_df)
        unique_files = merged_df['source_file'].nunique()
        
        print(f"🚨 Found Total {total_errors} error rows from {unique_files} files.")

        # 저장 경로 생성
        json_path = os.path.join(SAVE_DIR, OUTPUT_JSON_NAME)
        csv_path = os.path.join(SAVE_DIR, OUTPUT_CSV_NAME)

        # (1) JSON 저장
        merged_dict = merged_df.to_dict(orient='records')
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(merged_dict, f, indent=4, ensure_ascii=False)

        # (2) CSV 저장
        merged_df.to_csv(csv_path, index=False, encoding='utf-8-sig')

        print(f"💾 Merged files saved to:\n  - JSON: {json_path}\n  - CSV:  {csv_path}")
        
    else:
        print(f"✅ No errors found! All labels properly end with '{TARGET_TOKEN}'.")
    
    print("="*80)

if __name__ == "__main__":
    save_merged_error_samples()

🔍 Searching for CSV files in: /home/jovyan/CHJ/Mol-LLM_Custom/Inference_log/Benchmark_inference_csv ...
found 20 CSV files to check.



Scanning files: 100%|██████████| 20/20 [00:00<00:00, 32.65it/s]


🚨 Found Total 229 error rows from 8 files.
💾 Merged files saved to:
  - JSON: /home/jovyan/CHJ/Mol-LLM_Custom/utils/all_missing_eot_errors.json
  - CSV:  /home/jovyan/CHJ/Mol-LLM_Custom/utils/all_missing_eot_errors.csv


# 실제 데이터 verification

In [1]:
from datasets import load_from_disk
from collections import Counter
import pandas as pd
import os
import glob
from tqdm import tqdm

test_ds = load_from_disk('/home/jovyan/CHJ/Mol-LLM_Custom/dataset/train_official/GSAI-ML-LLaDA-8B-Instruct_string+graph_q32_test_512_Truncation')
test_chebi_text2mol_ds = test_ds.filter(lambda x: x['task'] == 'chebi-20-text2mol')

Filter:   0%|          | 0/32595 [00:00<?, ? examples/s]

In [2]:
test_chebi_text2mol_ds[0]

{'x': [[5, 0, 4, 5, 3, 0, 2, 0, 0], [5, 0, 4, 5, 3, 0, 2, 0, 0]],
 'edge_index': [[0, 1], [1, 0]],
 'edge_attr': [[0, 0, 0], [0, 0, 0]],
 'label': '<SELFIES>[C][O][C][=Branch1][C][=O][/C][=C][/C][C][C@H1][C@@H1][C][C][C][=C][C][=Branch1][C][=O][C][C][C@][Ring1][#Branch1][Branch1][C][C][C@H1][Ring1][N][C][=Branch1][C][=O][C][C@][Ring2][Ring1][Ring2][Ring1][P][C]</SELFIES>',
 'input_mol_string': '<SELFIES> <None> </SELFIES>',
 'task_subtask_pair': 'chebi-20-text2mol/0',
 'instruction': 'Generate a molecule that fulfills the requirement: <DESCRIPTION>The molecule is a steroid ester that is methyl (17E)-pregna-4,17-dien-21-oate substituted by oxo groups at positions 3 and 11. It is a 3-oxo-Delta(4) steroid, an 11-oxo steroid, a steroid ester and a methyl ester. It derives from a hydride of a pregnane.</DESCRIPTION>',
 'additional_x': [[5, 0, 4, 5, 3, 0, 2, 0, 0], [5, 0, 4, 5, 3, 0, 2, 0, 0]],
 'additional_edge_index': [[0, 1], [1, 0]],
 'additional_edge_attr': [[0, 0, 0], [0, 0, 0]],
 'tas